In [25]:
# based on https://github.com/chkwon/TrafficAssignment.jl

include(joinpath("/home/jzh/Dropbox/Research/",
"Data-driven_estimation_inverse_optimization", 
"Experiments/InverseVIsTraffic/Julia_files/load_network_uni-class.jl"))

using Graphs

function create_graph(start_node, end_node)
    @assert Base.length(start_node)==Base.length(end_node)

    no_node = max(maximum(start_node), maximum(end_node))
    no_arc = Base.length(start_node)

    graph = simple_inclist(no_node)
    for i=1:no_arc
        add_edge!(graph, start_node[i], end_node[i])
    end
    return graph
end

function get_vector(state, origin, destination, link_dic)
    current = destination
    parent = -1
    x = zeros(Int, maximum(link_dic))

    while parent != origin
        parent = state.parents[current]

        link_idx = link_dic[parent,current]

        if link_idx != 0
            x[link_idx] = 1
        end

        current = parent
    end

    return x
end

ta_data = load_ta_network("East_Massachusetts_Jan_PM")

# unpacking data from ta_data
network_name = ta_data.network_name

number_of_zones = ta_data.number_of_zones
number_of_nodes = ta_data.number_of_nodes
first_thru_node = ta_data.first_thru_node
number_of_links = ta_data.number_of_links

start_node = ta_data.start_node
end_node = ta_data.end_node
capacity = ta_data.capacity
link_length = ta_data.link_length

free_flow_time = ta_data.free_flow_time
B = ta_data.B
power = ta_data.power
speed_limit = ta_data.speed_limit
toll = ta_data.toll
link_type = ta_data.link_type
number_of_zones = ta_data.number_of_zones
total_od_flow = ta_data.total_od_flow
travel_demand = ta_data.travel_demand
od_pairs = ta_data.od_pairs

toll_factor = ta_data.toll_factor
distance_factor = ta_data.distance_factor

best_objective = ta_data.best_objective

# preparing a graph
graph = create_graph(start_node, end_node)
link_dic = sparse(start_node, end_node, 1:number_of_links);

In [26]:
travel_demand

8x8 Array{Float64,2}:
    0.0          0.0     1793.14       0.0     …    57.0467       0.0     
    0.001828     0.0     1130.3     2331.81         36.8403       4.93973 
  193.523      253.735      0.0       21.4761     5000.0          9.30787 
 1098.98      1278.74     433.081      0.0           0.004968  1726.84    
   83.5235      48.8645    49.2805    16.676         0.004922     0.004925
    9.8e-5       9.8e-5     9.8e-5  2725.25    …  1584.66       821.056   
 3041.72      1212.64      18.2178   572.201         0.0       1195.94    
    0.247283    14.3853  2736.91    1957.72          0.489529     0.0     

In [28]:
############
#Read in the demand file
file = open("East_Massachusetts_trips_Jan_PM.txt")
# demands = Dict{(Int64,Int64), Float64}()
s = 0
for line in eachline(file)
    if contains(line, "Origin")
        s = int(split(line)[2])
    else
        pairs = split(line, ";")
        for pair in pairs
            if !contains(pair, "\n")
                pair_vals = split(pair, ":")
                t, demand = int(pair_vals[1]), float(pair_vals[2])
#                 demands[(s,t)] = demand 
                travel_demand[s,t] = demand 
            end
        end
    end
end                
close(file)

In [29]:
travel_demand

8x8 Array{Float64,2}:
    0.0  1000.0  1000.0  5000.0  2000.0  3000.0   5000.0   8000.0
 1000.0     0.0  1000.0  2000.0  1000.0  4000.0   2000.0   4000.0
 1000.0  1000.0     0.0  2000.0  1000.0  3000.0   1000.0   2000.0
 5000.0  2000.0  2000.0     0.0  5000.0  4000.0   4000.0   7000.0
 2000.0  1000.0  1000.0  5000.0     0.0  2000.0   2000.0   5000.0
 3000.0  4000.0  3000.0  4000.0  2000.0     0.0   4000.0   8000.0
 5000.0  2000.0  1000.0  4000.0  2000.0  4000.0      0.0  10000.0
 8000.0  4000.0  2000.0  7000.0  5000.0  8000.0  10000.0      0.0

In [30]:
function MSA(coeffs) 
    polyEval(coeffs, pt) = sum([coeffs[i] * pt^(i-1) for i = 1:length(coeffs)]) 

    function BPR(x)
        bpr = similar(x)
        for i=1:length(bpr)
            bpr[i] = free_flow_time[i] * polyEval( coeffs, (x[i]/capacity[i]) ) 
        end
        return bpr
    end

    function all_or_nothing(travel_time)
        state = []
        path = []
        x = zeros(size(start_node))

        for r=1:size(travel_demand)[1]
            # for each origin node r, find shortest paths to all destination nodes
            state = dijkstra_shortest_paths(graph, travel_time, r)

            for s=1:size(travel_demand)[2]
                # for each destination node s, find the shortest-path vector
                # load travel demand
                x = x + travel_demand[r,s] * get_vector(state, r, s, link_dic)
            end
        end

        return x
    end

    # Finding a starting feasible solution
    travel_time = BPR(zeros(number_of_links))
    xl = all_or_nothing(travel_time)

    max_iter_no = 3e3
    l = 1
    #average_excess_cost = 1
    tol = 1e-6

    while l < max_iter_no
        l += 1

        xl_old = xl

        # Finding yl
        travel_time = BPR(xl)

        yl = all_or_nothing(travel_time)

        xl = xl + (yl - xl)/l

        xl_new = xl

        relative_gap = norm(xl_new - xl_old, 1) / norm(xl_new, 1)

        if relative_gap < tol
            break
        end
    end
    
    return xl
end

MSA (generic function with 1 method)

In [3]:
# getting the coefficients of the costs
coeffs_dict_Jan_PM = readall("../temp_files/coeffs_dict_Jan_AM.json")
coeffs_dict_Jan_PM = JSON.parse(coeffs_dict_Jan_PM)

xl = Dict{(Int64,Float64,Float64,Int64),Array{Float64,1}}()

deg_grid = [4:8]
c_grid = .5:.5:3.
lamb_grid = [10. .^(-3:4)]

for deg in deg_grid
    for c in c_grid
        for lam in lamb_grid
            coeffs_1 = coeffs_dict_Jan_PM["($(deg),$(c),$(lam),1)"]
            coeffs_2 = coeffs_dict_Jan_PM["($(deg),$(c),$(lam),2)"]
            coeffs_3 = coeffs_dict_Jan_PM["($(deg),$(c),$(lam),3)"]
#             xl[(deg, c, lam, 1)] = MSA(coeffs_1)
#             xl[(deg, c, lam, 2)] = MSA(coeffs_2)
#             xl[(deg, c, lam, 3)] = MSA(coeffs_3)
        end
    end
end

LoadError: key not found: "(4,0.5,0.001,1)"
while loading In[3], in expression starting on line 11

In [31]:
true_coeffs = [1, 0, 0, 0, .15]

xl = MSA(true_coeffs)

24-element Array{Float64,1}:
 12450.7 
 12097.0 
 12549.3 
 12903.0 
  8014.0 
  8010.33
 17442.0 
 17092.0 
 14970.7 
 15533.7 
 12591.0 
 12378.0 
 15404.3 
 15007.7 
  6010.67
  6013.0 
 17451.0 
 17497.7 
 19969.0 
 20133.0 
 14586.7 
 14376.0 
 26550.3 
 26503.7 

In [32]:
travel_demand

8x8 Array{Float64,2}:
    0.0  1000.0  1000.0  5000.0  2000.0  3000.0   5000.0   8000.0
 1000.0     0.0  1000.0  2000.0  1000.0  4000.0   2000.0   4000.0
 1000.0  1000.0     0.0  2000.0  1000.0  3000.0   1000.0   2000.0
 5000.0  2000.0  2000.0     0.0  5000.0  4000.0   4000.0   7000.0
 2000.0  1000.0  1000.0  5000.0     0.0  2000.0   2000.0   5000.0
 3000.0  4000.0  3000.0  4000.0  2000.0     0.0   4000.0   8000.0
 5000.0  2000.0  1000.0  4000.0  2000.0  4000.0      0.0  10000.0
 8000.0  4000.0  2000.0  7000.0  5000.0  8000.0  10000.0      0.0